#  一、多模态原理分析

In [ ]:
import torch;
import os;
import torch.backends.cudnn as cudnn # CUDA深度神经网络库后端

device = "cuda" if torch.cuda.is_available() else "cpu";


print(f"CUDA设备设置完成: {device} ✓ ");



print("pwdPath:", os.getcwd());


# 配置信息

   
# import os as _os
# import re as _re
# import sys as _sys
from dataclasses import dataclass
from typing import Optional




@dataclass
class ConfigParam:
    num_workers: int 
    valid_num_workers: int 
    train_data: str 
    val_data: str   
    use_augment: int
    vision_model: str # 视频模型
    # 视频模型
    #vision_model = "ViT-B-16";
    # 文本模型
    #text_model = "RoBERTa-wwm-ext-base-chinese";
    text_model:str 
    batch_size: int 
    valid_batch_size: int 
    seed: int 
    context_length: int
    wd:float #其他参数使用权重衰减（默认0.001）
    lr:float # 学习率
    # Adam的动量参数
        #   beta1: 一阶矩估计的衰减率（默认0.9），控制梯度历史的影响
        #   beta2: 二阶矩估计的衰减率（默认0.98或0.999），控制梯度平方历史的影响
    beta1:float # adam 
    beta2: float # adam 
    eps:float # 数值稳定性参数（默认1e-6或1e-8），防止除零
    max_steps: int # 最大步长
    accum_freq: int # 需要考虑梯度累积频率（accum_freq）
    max_epochs: int # 训练次数
    valid_epoch_interval:int
    valid_step_interval: int # 验证步数间隔
    #  保存定期检查点（根据save_epoch_frequency或最后一个epoch）
    save_epoch_frequency:int
    save_step_frequency: int # 保存步数频率
    # 创建余弦学习率调度器（带warmup）
    # cosine_lr函数会创建一个学习率调度器，学习率变化如下：
    #   1. Warmup阶段（前warmup步）：线性增长从0到lr
    #   2. Cosine衰减阶段（warmup步之后）：按余弦函数从lr衰减到0
    warmup: int # 
    precision:str # 
    local_device_rank: int # GPU index 
    mask_ratio: int # 
    resume: str # clip_cn_vit-b-16.pt 
    checkpoint_path:str
    logsfan:str 
    name: str 
    use_flash_attention: int
    # 恢复epoch和步数信息，并重新加载对应epoch的数据集
    reset_data_offset: int
    # 恢复优化器状态（如果未重置优化器）
    reset_optimizer: int
    # 是否加载教师模型（用于知识蒸馏）
    distillation:int 
    # 教师模型
    teacher_model_name: Optional[str] 
    freeze_vision:int
    # 新增字段
    aggregate: bool # 是否在计算损失前聚合跨GPU的特征
    gather_with_grad: bool # 是否在特征聚合时启用完整分布式梯度
    grad_checkpointing: bool # 是否启用梯度检查点
    kd_loss_weight: float # 知识蒸馏损失权重
    log_interval: int # 日志记录间隔
    report_training_batch_acc: bool # 是否报告训练批次准确率
    skip_aggregate: bool # 是否跳过聚合（aggregate = not skip_aggregate）
    skip_scheduler: bool # 是否跳过学习率衰减
    clip_weight_path: Optional[str] # CLIP预训练权重路径
    bert_weight_path: Optional[str] # BERT预训练权重路径
    device: str # 设备类型（cuda/cpu）
    rank: int # 分布式训练中的进程排名
    world_size: int # 分布式训练中的总进程数
    use_bn_sync: bool # 是否使用批归一化同步
   # log_path: str # 日志文件路径（运行时生成）
   # log_level: int # 日志级别
    should_save: bool # 是否是主节点
    



# from gettext import gettext as _, ngettext
# get_data函数会：
#   1. 加载LMDB格式的训练数据集（如果指定train_data）
#   2. 加载LMDB格式的验证数据集（如果指定val_data）
#   3. 创建DataLoader，支持多进程数据加载（num_workers）
#   4. 应用数据增强（如果启用use_augment）
#   5. 返回包含'train'和'val'键的字典
# epoch_id: 当前epoch编号，用于某些需要epoch相关数据增强的场景
# max_txt_length: 文本最大长度（token数），超过此长度的文本会被截断

# 上下文长度
#context_length = 52;

#config.append(["train_data", "../datapath/datasets/MUGE/lmdb/train"]);

config = ConfigParam(
    num_workers=0,  # 必须为0：LMDB Environment对象无法被pickle，多进程会导致错误 
    valid_num_workers=0,  # 必须为0：LMDB Environment对象无法被pickle，多进程会导致错误 
    train_data="../datapath/datasets/MUGE/lmdb/train",
    val_data="../datapath/datasets/MUGE/lmdb/valid",
    use_augment=True,
    vision_model="ViT-B-16",
    text_model="RoBERTa-wwm-ext-base-chinese",
    batch_size=48,
    valid_batch_size=48,
    seed=123,
    context_length=52,
    wd=0.001,
    lr=3e-06,
    beta1=0.9,
    beta2=0.98,
    eps=1e-06,
    max_steps=52150,
    accum_freq=1,
    max_epochs=10,
    valid_epoch_interval=1,
    valid_step_interval=1000,
    save_epoch_frequency=1,
    save_step_frequency=999999,
    warmup=100,
    precision="amp",
    local_device_rank=0,
    mask_ratio=0,
    resume="../datapath/pretrained_weights/clip_cn_vit-b-16.pt",
    checkpoint_path="../datapath/experiments/muge_finetune_vit-b-16_roberta-base_bs48_1gpu/checkpoints",
    logsfan="../datapath/experiments/",
    name="muge_finetune_vit-b-16_roberta-base_bs48_1gpu",
    use_flash_attention=False,
    reset_data_offset=True, #  恢复步长信息
    reset_optimizer=True, # 优化器
    distillation=False,
    teacher_model_name=None,
    freeze_vision=False,
    # 新增字段
    aggregate=False,
    gather_with_grad=False,
    grad_checkpointing=True,
    kd_loss_weight=0.5,
    log_interval=10,
    report_training_batch_acc=True,
    skip_aggregate=True,
    skip_scheduler=False,
    clip_weight_path=None,
    bert_weight_path=None,
    device="cuda",
    rank=0,
    world_size=1,
    use_bn_sync=False,
#    log_path="datapath/experiments/muge_finetune_vit-b-16_roberta-base_bs48_1gpu/out_2025-12-26-10-04-37.log",
#   log_level=20,
    should_save = True,
    );

# 设置检查点保存路径: {logsfan}/{name}/checkpoints/
# 所有模型检查点将保存在此目录下
config.checkpoint_path = os.path.join(config.logsfan, config.name, "checkpoints")
print(f"config:{config}");


CUDA设备设置完成: cuda ✓ 
pwdPath: d:\Work\AI\stable_diffusion\stable-diffusion-webui\Chinese-CLIP\cn_clip
config:ConfigParam(num_workers=0, valid_num_workers=0, train_data='../datapath/datasets/MUGE/lmdb/train', val_data='../datapath/datasets/MUGE/lmdb/valid', use_augment=True, vision_model='ViT-B-16', text_model='RoBERTa-wwm-ext-base-chinese', batch_size=48, valid_batch_size=48, seed=123, context_length=52, wd=0.001, lr=3e-06, beta1=0.9, beta2=0.98, eps=1e-06, max_steps=52150, accum_freq=1, max_epochs=10, valid_epoch_interval=1, valid_step_interval=1000, save_epoch_frequency=1, save_step_frequency=999999, warmup=100, precision='amp', local_device_rank=0, mask_ratio=0, resume='../datapath/pretrained_weights/clip_cn_vit-b-16.pt', checkpoint_path='../datapath/experiments/muge_finetune_vit-b-16_roberta-base_bs48_1gpu\\checkpoints', logsfan='../datapath/experiments/', name='muge_finetune_vit-b-16_roberta-base_bs48_1gpu', use_flash_attention=False, reset_data_offset=True, reset_optimizer=True, d

# 二、设置分布式训练模式


In [58]:



import torch.distributed as dist     # 分布式训练支持


dist_initialized = False;


if not dist.is_initialized():
    try:
        # 设置环境变量（如果未设置）
        if 'MASTER_ADDR' not in os.environ:
            os.environ['MASTER_ADDR'] = '127.0.0.1'
        if 'MASTER_PORT' not in os.environ:
            os.environ['MASTER_PORT'] = '29501'
        
        # 尝试使用 gloo 后端，使用 tcp 初始化方法
        print(f"尝试使用 GLOO 后端 (tcp://{os.environ.get('MASTER_ADDR')}:{os.environ.get('MASTER_PORT')})...")
        
        dist.init_process_group(
            backend="gloo",
            init_method=f"tcp://{os.environ.get('MASTER_ADDR')}:{os.environ.get('MASTER_PORT')}",
            rank=0,
            world_size=1
        )
        dist_initialized = True
        print("✓ 使用 GLOO 后端初始化单进程分布式进程组成功")
    except Exception as e:
        # 如果初始化失败，设置 aggregate=False 并跳过分布式初始化
        print(f"无法初始化分布式进程组: {e}")
        print("提示: Windows 上的 GLOO 后端可能不可用，将使用单节点模式（跳过聚合）")
        print("设置 aggregate=False 以避免使用分布式函数")
        # args.aggregate = False  # 禁用聚合，避免使用分布式函数
        # args.skip_aggregate = True
        # dist_initialized = False
else:
    dist_initialized = True
    print("分布式进程组已初始化，跳过")




# if dist_initialized:
#     args.rank = dist.get_rank()
#     args.world_size = dist.get_world_size()
# else:
#     # 如果分布式未初始化，设置默认值
#     args.rank = 0
#     args.world_size = 1

尝试使用 GLOO 后端 (tcp://127.0.0.1:29501)...
无法初始化分布式进程组: makeDeviceForHostname(): unsupported gloo device
提示: Windows 上的 GLOO 后端可能不可用，将使用单节点模式（跳过聚合）
设置 aggregate=False 以避免使用分布式函数


# 三、 设置输出路径和日志系统



In [59]:
import time;            # 时间处理
from time import gmtime, strftime;  # 时间格式化
import logging;          # 日志记录


# ============================================================================
# Chinese-CLIP项目内部模块导入
# ============================================================================
# from cn_clip.clip import load  # 模型权重加载函数
# from cn_clip.clip.model import convert_weights, convert_state_dict, resize_pos_embed, CLIP  # CLIP模型定义
# from cn_clip.training.train import train, evaluate  # 训练和评估函数
# from cn_clip.training.data import get_data  # 数据加载器构建
# from cn_clip.training.params import parse_args  # 命令行参数解析
from cn_clip.training.logger import setup_primary_logging, setup_worker_logging  # 日志系统设置
# from cn_clip.training.scheduler import cosine_lr  # 余弦学习率调度器





rank = 0;
# 设置日志路径啦
# D:\Work\AI\stable_diffusion\stable-diffusion-webui\Chinese-CLIP\datapath\experiments\muge_finetune_vit-b-16_roberta-base_bs48_1gpu\out_2025-12-26-10-12-26.log
# logsfan=os.getcwd()+"/../datapath/experiments/";
# log_fix_name = "muge_finetune_vit-b-16_roberta-base_bs48_1gpu";

time_suffix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
#print('')
# 设置日志文件路径: {logsfan}/{name}/out_{时间戳}.log
# 例如: datapath/experiments/muge_finetune_vit-b-16_roberta-base_bs128_8gpu/out_2024-12-25-16-30-00.log
g_log_path = os.path.join(config.logsfan, config.name, "chensong_out_{}.log".format(time_suffix))



# 只有主进程（rank=0）创建输出目录，避免多进程重复创建
# 在分布式训练中，所有进程共享文件系统，只需主进程创建目录即可
#if is_master(args):
if True:
    for dirname in [config.checkpoint_path]:
        if dirname:
            os.makedirs(dirname, exist_ok=True)  # exist_ok=True表示目录已存在时不报错    

# 验证精度设置（混合精度/FP16/FP32）
# - 'amp': 自动混合精度（Automatic Mixed Precision），推荐使用，平衡速度和精度
# - 'fp16': 纯FP16精度，速度最快但可能影响精度
# - 'fp32': 纯FP32精度，精度最高但速度最慢
#assert args.precision in ['amp', 'fp16', 'fp32']

# 设置日志级别
# DEBUG: 详细调试信息（包括所有中间变量值）
# INFO: 一般信息（训练进度、损失值等）
log_level = logging.DEBUG;

# 设置主进程日志系统
# 主进程（rank=0）负责将日志写入文件
# 返回一个队列，用于接收工作进程的日志消息
log_queue = setup_primary_logging(g_log_path, log_level, rank);

# 设置工作进程日志系统
# 工作进程（rank>0）通过队列将日志消息发送给主进程
# 主进程统一写入文件，避免多进程同时写文件造成冲突
setup_worker_logging(rank, log_queue, log_level)
logging.info("✓ 日志系统设置完成")

2025-12-27,15:49:52 | INFO | Rank 0 | ✓ 日志系统设置完成


# 四、构建CLIP模型

In [60]:
from pathlib import Path;  # 路径处理



# 1. 加载视觉模型配置文件
# 配置文件路径: cn_clip/clip/model_configs/{vision_model}.json
# 例如: ViT-B-16 -> ViT-B-16.json, ViT-L/14 -> ViT-L-14.json
# 配置文件包含：embed_dim, image_resolution, vision_layers, vision_width等
vision_model_config_file = Path(os.getcwd()) / f"clip/model_configs/{config.vision_model.replace('/', '-')}.json"
logging.info(f'Loading vision model config from {vision_model_config_file}')
assert os.path.exists(vision_model_config_file), f"Vision model config not found: {vision_model_config_file}"


# 2. 加载文本模型配置文件
# 配置文件路径: cn_clip/clip/model_configs/{text_model}.json
# 例如: RoBERTa-wwm-ext-base-chinese -> RoBERTa-wwm-ext-base-chinese.json
# 配置文件包含：vocab_size, text_hidden_size, text_num_layers等
text_model_config_file = Path(os.getcwd())/ f"clip/model_configs/{config.text_model.replace('/', '-')}.json"
logging.info(f'Loading text model config from {text_model_config_file}')
assert os.path.exists(text_model_config_file), f"Text model config not found: {text_model_config_file}"

2025-12-27,15:49:55 | INFO | Rank 0 | Loading vision model config from d:\Work\AI\stable_diffusion\stable-diffusion-webui\Chinese-CLIP\cn_clip\clip\model_configs\ViT-B-16.json
2025-12-27,15:49:55 | INFO | Rank 0 | Loading text model config from d:\Work\AI\stable_diffusion\stable-diffusion-webui\Chinese-CLIP\cn_clip\clip\model_configs\RoBERTa-wwm-ext-base-chinese.json


In [61]:
import json;            # JSON配置文件解析
# 合并视觉和文本模型配置
# 将两个JSON配置文件的内容合并为一个字典，用于创建CLIP模型
with open(vision_model_config_file, 'r') as fv, open(text_model_config_file, 'r') as ft:
    model_info = json.load(fv)  # 加载视觉模型配置（字典格式）
    
    # 处理vision_layers字段
    # 某些配置文件中vision_layers可能是字符串格式（如"[12, 12, 12, 12]"）
    # 需要转换为Python列表格式
    if isinstance(model_info['vision_layers'], str):
        model_info['vision_layers'] = eval(model_info['vision_layers']);
    
    # 合并文本模型配置到model_info
    # 文本模型的配置项会添加到model_info中，如果键名相同会被覆盖
    for k, v in json.load(ft).items():
        model_info[k] = v

# 添加FlashAttention使用标志
# FlashAttention是一种优化的注意力计算方式，可以加速训练并降低显存占用
model_info['use_flash_attention'] = config.use_flash_attention;



# logging.debug(f"model_info:{model_info}");


model_info

{'embed_dim': 512,
 'image_resolution': 224,
 'vision_layers': 12,
 'vision_width': 768,
 'vision_patch_size': 16,
 'vocab_size': 21128,
 'text_attention_probs_dropout_prob': 0.1,
 'text_hidden_act': 'gelu',
 'text_hidden_dropout_prob': 0.1,
 'text_hidden_size': 768,
 'text_initializer_range': 0.02,
 'text_intermediate_size': 3072,
 'text_max_position_embeddings': 512,
 'text_num_attention_heads': 12,
 'text_num_hidden_layers': 12,
 'text_type_vocab_size': 2,
 'use_flash_attention': False}

In [62]:
# ============================================================================
# Chinese-CLIP项目内部模块导入
# ============================================================================
from cn_clip.clip import load  # 模型权重加载函数
from cn_clip.clip.model import convert_weights, convert_state_dict, resize_pos_embed, CLIP  # CLIP模型定义
from cn_clip.training.train import train, evaluate  # 训练和评估函数
from cn_clip.training.data import get_data  # 数据加载器构建
from cn_clip.training.params import parse_args  # 命令行参数解析
from cn_clip.training.logger import setup_primary_logging, setup_worker_logging  # 日志系统设置
from cn_clip.training.scheduler import cosine_lr  # 余弦学习率调度器 
# 创建CLIP模型实例
# CLIP模型包含两个编码器：
#   - visual: 视觉编码器（ViT或ResNet），用于编码图像
#   - textual: 文本编码器（RoBERTa），用于编码文本
#   两个编码器将图像和文本映射到同一个特征空间，用于计算相似度
model = CLIP(**model_info)



model

CLIP(
  (visual): VisualTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

In [63]:
 # 加载预训练权重（如果指定）
# clip_weight_path: CLIP模型的预训练权重路径（包含视觉和文本编码器）
clip_weight_path = None;

# bert_weight_path: 仅文本编码器的预训练权重路径（可选，用于单独加载BERT权重）
bert_weight_path = None;
# if args.clip_weight_path is not None:
#     assert os.path.exists(args.clip_weight_path), "Pretrained CLIP weight not exists!"
# if args.bert_weight_path is not None:
#     assert os.path.exists(args.bert_weight_path), "Pretrained BERT weight not exists!"
# 加载权重到模型
# load函数会：
#   1. 加载CLIP权重（如果指定clip_weight_path）
#   2. 加载BERT权重（如果指定bert_weight_path）
#   3. 处理权重名称不匹配的情况（如添加/移除"module."前缀）
#   4. 如果使用FlashAttention，转换权重格式
new_model = load(model, clip_path=clip_weight_path, bert_path=bert_weight_path, use_flash_attention=config.use_flash_attention);
logging.info("CLIP模型构建完成 ✓ ");


new_model

2025-12-27,15:50:06 | INFO | Rank 0 | CLIP模型构建完成 ✓ 


CLIP(
  (visual): VisualTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

# 五、模型精度和优化设置

In [65]:





def convert_models_to_fp32(model):
    """
    将模型的所有参数和梯度转换为FP32格式
    
    参数:
        model: PyTorch模型
    说明:
        用于混合精度训练（AMP）或FP32训练时，确保模型参数为FP32格式
        参考: https://github.com/openai/CLIP/issues/83
    """
    for p in model.parameters():
        p.data = p.data.float()
        if p.grad:
            p.grad.data = p.grad.data.float()





# 验证精度设置（混合精度/FP16/FP32）
    # - 'amp': 自动混合精度（Automatic Mixed Precision），推荐使用，平衡速度和精度
    # - 'fp16': 纯FP16精度，速度最快但可能影响精度
    # - 'fp32': 纯FP32精度，精度最高但速度最慢
# precision = "amp";

if config.precision == "amp" or config.precision == "fp32":
    convert_models_to_fp32(model)



# 将模型移动到指定的GPU设备
model.cuda(device);

# 对于FP16训练，将模型权重转换为FP16格式
if config.precision == "fp16":
    convert_weights(model);



def torch_version_str_compare_lessequal(version1, version2):
    """
    比较两个PyTorch版本号，判断version1是否小于等于version2
    
    参数:
        version1: 版本号字符串，如 "1.8.0"
        version2: 版本号字符串，如 "2.0.0"
    返回:
        bool: 如果version1 <= version2返回True，否则返回False
    说明:
        用于检查PyTorch版本兼容性，例如梯度检查点功能需要PyTorch >= 1.8.0
    """
    # 解析版本号，忽略"+"后的构建信息（如"1.8.0+cu111"）
    v1 = [int(entry) for entry in version1.split("+")[0].split(".")]
    v2 = [int(entry) for entry in version2.split("+")[0].split(".")]
    assert len(v1) == 3, "Cannot parse the version of your installed pytorch! ({})".format(version1)
    assert len(v2) == 3, "Illegal version specification ({}). Should be in 1.X.Y format.".format(version2)
    return sorted([v1, v2])[0] == v1


# 梯度检查点：用计算时间换显存，适合显存不足的情况

assert not torch_version_str_compare_lessequal(torch.__version__, "1.8.0"), \
    "Currently our grad_checkpointing is not compatible with torch version <= 1.8.0."
model.set_grad_checkpointing();
logging.info("Grad-checkpointing activated.");



# 同步批归一化：在分布式训练中同步BN统计量
#if args.use_bn_sync:
#model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(model)


2025-12-27,15:50:14 | INFO | Rank 0 | Grad-checkpointing activated.


# 六、单节点训练配置

In [66]:
from model_wrapper import ModelWrapper;
# 单节点单GPU训练：直接将模型移动到GPU，不使用DistributedDataParallel
# 这样可以避免DDP的开销，简化训练流程
# 注意：train.py 中使用了 model.module，所以我们需要创建一个兼容的包装类
# 或者直接使用模型（需要修改 train.py 中的 model.module 引用）

# 将模型移动到指定GPU设备
model = model.to(device);


 



  # 包装模型以兼容 train.py 中的 model.module 引用
model = ModelWrapper(model);


# 如果使用FP16，转换权重
if config.precision == "fp16":
    convert_weights(model._model);


logging.info(f"单节点训练配置完成（模型已移动到 {device}）✓ ")








2025-12-27,15:50:17 | INFO | Rank 0 | 单节点训练配置完成（模型已移动到 cuda）✓ 


# 七、初始化数据集和数据加载器

In [67]:

     
 
data = get_data(config, epoch_id=0, max_txt_length=config.context_length);


logging.debug(f"data:{data}");

logging.info(" 数据集和数据加载器初始化完成 ✓");

2025-12-27,15:50:20 | INFO | Rank 0 | train LMDB file contains 129380 images and 250314 pairs.
2025-12-27,15:50:20 | INFO | Rank 0 | val LMDB file contains 29806 images and 30588 pairs.
2025-12-27,15:50:20 | DEBUG | Rank 0 | data:{'train': DataInfo(dataloader=<torch.utils.data.dataloader.DataLoader object at 0x0000028D9F50C520>, sampler=<torch.utils.data.sampler.RandomSampler object at 0x0000028D9F50D750>, dataset=<cn_clip.training.data.LMDBDataset object at 0x0000028D9F114D00>, epoch_id=0), 'val': DataInfo(dataloader=<torch.utils.data.dataloader.DataLoader object at 0x0000028D9F50DC90>, sampler=<torch.utils.data.sampler.RandomSampler object at 0x0000028D9F50FF70>, dataset=<cn_clip.training.data.LMDBDataset object at 0x0000028D9F4EB490>, epoch_id=0)}
2025-12-27,15:50:20 | INFO | Rank 0 |  数据集和数据加载器初始化完成 ✓


# 八、初始化优化器和学习率调整器

In [68]:
# 定义参数分组规则（用于不同的权重衰减策略）
# 为什么需要分组？
#   - BatchNorm/LayerNorm的scale和bias参数通常不使用权重衰减
#   - 偏置（bias）参数通常不使用权重衰减
#   - logit_scale（温度参数）通常不使用权重衰减
#   这样可以提高训练稳定性和最终性能
exclude = lambda n : "bn" in n or "ln" in n or "bias" in n or 'logit_scale' in n;


logging.debug(f"exclude:{exclude}");



include = lambda n : not exclude(n);

logging.debug(f"include:{include}");



# 获取模型所有参数并按规则分组
# named_parameters: 返回(参数名, 参数张量)的迭代器
# requires_grad=True: 只选择需要梯度的参数（排除冻结的参数）
named_parameters = list(model.named_parameters());


named_parameters



#logging.debug(f"named_parameters:{named_parameters}");





2025-12-27,15:50:22 | DEBUG | Rank 0 | exclude:<function <lambda> at 0x0000028ABCAE6710>
2025-12-27,15:50:22 | DEBUG | Rank 0 | include:<function <lambda> at 0x0000028D9F40D2D0>


[('text_projection',
  Parameter containing:
  tensor([[ 0.0741, -0.0306,  0.0453,  ..., -0.0077, -0.0049,  0.0210],
          [-0.0740,  0.0169,  0.0240,  ...,  0.0061,  0.0537,  0.0245],
          [-0.0286, -0.0144,  0.0485,  ..., -0.0580, -0.0638,  0.0006],
          ...,
          [ 0.0197, -0.0190, -0.0152,  ..., -0.0168,  0.0006, -0.0422],
          [ 0.0676, -0.0147, -0.0625,  ...,  0.0581,  0.0084,  0.0266],
          [-0.0505,  0.0052,  0.0109,  ...,  0.0020,  0.0464, -0.0192]],
         device='cuda:0', requires_grad=True)),
 ('logit_scale',
  Parameter containing:
  tensor(2.6593, device='cuda:0', requires_grad=True)),
 ('visual.class_embedding',
  Parameter containing:
  tensor([ 5.7351e-02,  8.1506e-03,  2.6840e-03, -1.8827e-02, -1.9833e-02,
           2.6450e-02, -1.4397e-02, -4.9995e-02,  6.0167e-02, -9.1780e-02,
          -2.3844e-02,  1.5948e-02,  4.6775e-02, -4.3686e-02, -2.1456e-02,
          -1.4117e-02, -3.0200e-02,  6.6943e-02, -2.6163e-02,  2.8524e-02,
          

In [69]:
test_arr = [3, 4, 5, 6, 7, 45, 453, 34, 23, 45, 45, 65, 45, 90];
 
for   p in test_arr:
    logging.debug(f"  p:{p}");
    # test_out.append(int(p));  # Convert string to integer

# logging.debug(f"test_arr: {test_arr}");
# logging.debug(f"test_out: {test_out}");

2025-12-27,15:50:41 | DEBUG | Rank 0 |   p:3
2025-12-27,15:50:41 | DEBUG | Rank 0 |   p:4
2025-12-27,15:50:41 | DEBUG | Rank 0 |   p:5
2025-12-27,15:50:41 | DEBUG | Rank 0 |   p:6
2025-12-27,15:50:41 | DEBUG | Rank 0 |   p:7
2025-12-27,15:50:41 | DEBUG | Rank 0 |   p:45
2025-12-27,15:50:41 | DEBUG | Rank 0 |   p:453
2025-12-27,15:50:41 | DEBUG | Rank 0 |   p:34
2025-12-27,15:50:41 | DEBUG | Rank 0 |   p:23
2025-12-27,15:50:41 | DEBUG | Rank 0 |   p:45
2025-12-27,15:50:41 | DEBUG | Rank 0 |   p:45
2025-12-27,15:50:41 | DEBUG | Rank 0 |   p:65
2025-12-27,15:50:41 | DEBUG | Rank 0 |   p:45
2025-12-27,15:50:41 | DEBUG | Rank 0 |   p:90


In [70]:
# 不使用权重衰减的参数组（BN/LN/bias/logit_scale）
gain_or_bias_params = [p for n, p in named_parameters if exclude(n) and p.requires_grad];



logging.debug(f"gain_or_bias_params:{gain_or_bias_params}");



2025-12-27,15:50:48 | DEBUG | Rank 0 | gain_or_bias_params:[Parameter containing:
tensor(2.6593, device='cuda:0', requires_grad=True), Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1.,

In [71]:
from torch import optim              # 优化器（AdamW等）
from torch.cuda.amp import GradScaler  # 混合精度训练的梯度缩放器
from math import ceil
# 使用权重衰减的参数组（其他所有参数）
rest_params = [p for n, p in named_parameters if include(n) and p.requires_grad];

logging.debug(f"rest_params:{rest_params}");






if config.train_data is None:
    # 如果没有训练数据，则不创建优化器和调度器（仅用于评估）
    optimizer = None;
    scheduler = None;
else:
    # 创建AdamW优化器，对不同参数组应用不同的权重衰减
    # AdamW是Adam的改进版本，使用解耦的权重衰减（decoupled weight decay）
    # 相比Adam，AdamW通常能获得更好的泛化性能
    optimizer = optim.AdamW(
        [
            {"params": gain_or_bias_params, "weight_decay": 0.},  # BN/LN/bias/logit_scale不使用权重衰减
            {"params": rest_params, "weight_decay": config.wd},     # 其他参数使用权重衰减（默认0.001）
        ],
        lr=config.lr,                    # 初始学习率（peak learning rate）
        betas=(config.beta1, config.beta2), # Adam的动量参数
        #   beta1: 一阶矩估计的衰减率（默认0.9），控制梯度历史的影响
        #   beta2: 二阶矩估计的衰减率（默认0.98或0.999），控制梯度平方历史的影响
        eps=config.eps,                   # 数值稳定性参数（默认1e-6或1e-8），防止除零
    );
    
    # 计算总训练步数
    # num_batches: 一个epoch的批次数（考虑分布式训练，每个进程只看到部分数据）
    num_batches = data["train"].dataloader.num_batches;
    
    if config.max_steps is not None:
        # 如果指定了最大步数（max_steps），根据步数计算需要的epoch数
        # 注意：需要考虑梯度累积频率（accum_freq）
        # 例如：如果max_steps=1000, accum_freq=2, num_batches=500
        #   则实际需要: ceil(1000 * 2 / 500) = ceil(4) = 4个epoch
        config.max_epochs = ceil(config.max_steps * config.accum_freq / num_batches);
    else:
        # 如果指定了最大epoch数（max_epochs），根据epoch数计算总步数
        # 例如：如果max_epochs=3, num_batches=500, accum_freq=2
        #   则总步数: (500 // 2) * 3 = 250 * 3 = 750步
        assert config.max_epochs is not None and config.max_epochs > 0;
        config.max_steps = (num_batches // config.accum_freq) * config.max_epochs;
    
    total_steps = config.max_steps;
    
    # 创建余弦学习率调度器（带warmup）
    # cosine_lr函数会创建一个学习率调度器，学习率变化如下：
    #   1. Warmup阶段（前warmup步）：线性增长从0到lr
    #   2. Cosine衰减阶段（warmup步之后）：按余弦函数从lr衰减到0
    # 这种调度策略通常能获得更好的训练效果
    scheduler = cosine_lr(optimizer, config.lr, config.warmup, total_steps);


# 创建梯度缩放器（用于混合精度训练AMP）
# GradScaler用于解决FP16训练中的梯度下溢问题
# 工作原理：
#   1. 在反向传播前，将损失值乘以一个缩放因子（scale）
#   2. 反向传播后，梯度也会被缩放
#   3. 如果梯度没有溢出，将梯度除以缩放因子恢复原值
#   4. 如果梯度溢出，跳过本次更新并增大缩放因子
# 注意：只有使用AMP（自动混合精度）时才需要scaler
scaler = GradScaler() if config.precision == "amp" else None;





logging.info(f"优化器和学习率调度器初始化完成 ✓  scaler:{scaler}");



C:\Users\chen_song\AppData\Local\Temp\ipykernel_26340\1364849347.py:69: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler() if config.precision == "amp" else None;
2025-12-27,15:50:55 | DEBUG | Rank 0 | rest_params:[Parameter containing:
tensor([[ 0.0741, -0.0306,  0.0453,  ..., -0.0077, -0.0049,  0.0210],
        [-0.0740,  0.0169,  0.0240,  ...,  0.0061,  0.0537,  0.0245],
        [-0.0286, -0.0144,  0.0485,  ..., -0.0580, -0.0638,  0.0006],
        ...,
        [ 0.0197, -0.0190, -0.0152,  ..., -0.0168,  0.0006, -0.0422],
        [ 0.0676, -0.0147, -0.0625,  ...,  0.0581,  0.0084,  0.0266],
        [-0.0505,  0.0052,  0.0109,  ...,  0.0020,  0.0464, -0.0192]],
       device='cuda:0', requires_grad=True), Parameter containing:
tensor([ 5.7351e-02,  8.1506e-03,  2.6840e-03, -1.8827e-02, -1.9833e-02,
         2.6450e-02, -1.4397e-02, -4.9995e-02,  6.0167e-02, -9.1780e-02,
        -2.3844e

# 九、记录和保存超参数

In [72]:

# 主进程记录超参数到日志
logging.debug("master Params:") 
for name in sorted(vars(config)):
    val = getattr(config, name);
    logging.debug(f"{name}: {val}"); 

logging.debug(f"Use GPU: {config.local_device_rank} for training")
logging.debug("✓ 超参数记录完成")

# 关于mask_ratio的提示（FLIP策略仅支持ViT，不支持ResNet）
if config.mask_ratio > 0 and config.vision_model in ['RN50']:
    logging.debug("Note: mask_ratio > 0 (FLIP strategy) is currently only implemented for VisualTransformer. " + \
        "It will not function for ResNet backbone.")    

2025-12-27,15:51:31 | DEBUG | Rank 0 | master Params:
2025-12-27,15:51:31 | DEBUG | Rank 0 | accum_freq: 1
2025-12-27,15:51:31 | DEBUG | Rank 0 | aggregate: False
2025-12-27,15:51:31 | DEBUG | Rank 0 | batch_size: 48
2025-12-27,15:51:31 | DEBUG | Rank 0 | bert_weight_path: None
2025-12-27,15:51:31 | DEBUG | Rank 0 | beta1: 0.9
2025-12-27,15:51:31 | DEBUG | Rank 0 | beta2: 0.98
2025-12-27,15:51:31 | DEBUG | Rank 0 | checkpoint_path: ../datapath/experiments/muge_finetune_vit-b-16_roberta-base_bs48_1gpu\checkpoints
2025-12-27,15:51:31 | DEBUG | Rank 0 | clip_weight_path: None
2025-12-27,15:51:31 | DEBUG | Rank 0 | context_length: 52
2025-12-27,15:51:31 | DEBUG | Rank 0 | device: cuda
2025-12-27,15:51:31 | DEBUG | Rank 0 | distillation: False
2025-12-27,15:51:31 | DEBUG | Rank 0 | eps: 1e-06
2025-12-27,15:51:31 | DEBUG | Rank 0 | freeze_vision: False
2025-12-27,15:51:31 | DEBUG | Rank 0 | gather_with_grad: False
2025-12-27,15:51:31 | DEBUG | Rank 0 | grad_checkpointing: True
2025-12-27,15:

2025-12-27,15:51:31 | DEBUG | Rank 0 | mask_ratio: 0
2025-12-27,15:51:31 | DEBUG | Rank 0 | max_epochs: 10
2025-12-27,15:51:31 | DEBUG | Rank 0 | max_steps: 52150
2025-12-27,15:51:31 | DEBUG | Rank 0 | name: muge_finetune_vit-b-16_roberta-base_bs48_1gpu
2025-12-27,15:51:31 | DEBUG | Rank 0 | num_workers: 0
2025-12-27,15:51:31 | DEBUG | Rank 0 | precision: amp
2025-12-27,15:51:31 | DEBUG | Rank 0 | rank: 0
2025-12-27,15:51:31 | DEBUG | Rank 0 | report_training_batch_acc: True
2025-12-27,15:51:31 | DEBUG | Rank 0 | reset_data_offset: True
2025-12-27,15:51:31 | DEBUG | Rank 0 | reset_optimizer: True
2025-12-27,15:51:31 | DEBUG | Rank 0 | resume: ../datapath/pretrained_weights/clip_cn_vit-b-16.pt
2025-12-27,15:51:31 | DEBUG | Rank 0 | save_epoch_frequency: 1
2025-12-27,15:51:31 | DEBUG | Rank 0 | save_step_frequency: 999999
2025-12-27,15:51:31 | DEBUG | Rank 0 | seed: 123
2025-12-27,15:51:31 | DEBUG | Rank 0 | skip_aggregate: True
2025-12-27,15:51:31 | DEBUG | Rank 0 | skip_scheduler: Fals

# 十、加载检查点（如果存在）

In [73]:
logging.debug("步骤10: 加载检查点（如果存在）...")
start_epoch = 0  # 起始epoch
steps = 0        # 起始步数

# 如果未指定resume路径，自动查找最新的检查点
if config.resume is None:
    latest_path = os.path.join(config.checkpoint_path, f"epoch_latest.pt")
    if os.path.isfile(latest_path):
        config.resume = latest_path


if config.resume is not None:
    if os.path.isfile(config.resume):
        logging.info(f"=> begin to load checkpoint '{config.resume}'")
        
        # 加载检查点（加载到CPU，避免GPU显存问题）
        checkpoint = torch.load(config.resume, map_location="cpu")
        
        # 过滤掉bert.pooler层的参数（某些版本可能不兼容）
        sd = {k: v for k, v in checkpoint["state_dict"].items() if "bert.pooler" not in k}
        
        # 处理键名中的 "module." 前缀（从DDP模型保存的检查点）
        # 检查点可能包含 "module." 前缀，但我们的 ModelWrapper 模型没有这个前缀
        if sd and next(iter(sd.keys())).startswith("module."):
            # 移除 "module." 前缀
            sd = {k[len("module."):]: v for k, v in sd.items()}
            prefix_for_resize = ""  # 移除前缀后，不需要前缀
        else:
            prefix_for_resize = ""  # 没有前缀
        
        # 如果位置编码大小不匹配，通过插值调整
        resize_pos_embed(sd, model, prefix=prefix_for_resize)
        
        # 如果使用FlashAttention，转换状态字典格式
        if config.use_flash_attention:
            sd = convert_state_dict(sd)
        
        # 加载模型权重
        # 对于 ModelWrapper，需要加载到 _model
        if hasattr(model, '_model'):
            model._model.load_state_dict(sd, strict=False)
        else:
            model.load_state_dict(sd, strict=False)
        
        # 恢复epoch和步数信息，并重新加载对应epoch的数据集
        if not config.reset_data_offset:
            start_epoch = checkpoint["epoch"]
            steps = checkpoint["step"]
            data = get_data(config, 
                            epoch_id=start_epoch, 
                            max_txt_length=config.context_length)
        
        # 恢复优化器状态（如果未重置优化器）
        if not config.reset_optimizer and optimizer is not None:
            optimizer.load_state_dict(checkpoint["optimizer"])
            logging.info("=> optimizer state is restored from the checkpoint")
        
        logging.info(
            f"=> loaded checkpoint '{config.resume}' (epoch {checkpoint['epoch']} @ {steps} steps)"
        )
        logging.info(f"✓ 检查点加载完成: epoch {checkpoint['epoch']} @ {steps} steps")
    else:
        logging.info("=> no checkpoint found at '{}'".format(config.resume))
        logging.info("未找到检查点，从头开始训练")
else:
    logging.info("未指定检查点路径，从头开始训练")



data 

2025-12-27,15:51:34 | DEBUG | Rank 0 | 步骤10: 加载检查点（如果存在）...
2025-12-27,15:51:34 | INFO | Rank 0 | => begin to load checkpoint '../datapath/pretrained_weights/clip_cn_vit-b-16.pt'


{'train': DataInfo(dataloader=<torch.utils.data.dataloader.DataLoader object at 0x0000028D9F50C520>, sampler=<torch.utils.data.sampler.RandomSampler object at 0x0000028D9F50D750>, dataset=<cn_clip.training.data.LMDBDataset object at 0x0000028D9F114D00>, epoch_id=0),
 'val': DataInfo(dataloader=<torch.utils.data.dataloader.DataLoader object at 0x0000028D9F50DC90>, sampler=<torch.utils.data.sampler.RandomSampler object at 0x0000028D9F50FF70>, dataset=<cn_clip.training.data.LMDBDataset object at 0x0000028D9F4EB490>, epoch_id=0)}

2025-12-27,15:51:34 | INFO | Rank 0 | => loaded checkpoint '../datapath/pretrained_weights/clip_cn_vit-b-16.pt' (epoch 15 @ 0 steps)
2025-12-27,15:51:34 | INFO | Rank 0 | ✓ 检查点加载完成: epoch 15 @ 0 steps


# 十一、CUDA优化设置

In [74]:
import torch.backends.cudnn as cudnn # CUDA深度神经网络库后端
cudnn.benchmark = True;        # 启用CUDNN自动调优，加速训练（但可能降低可复现性）
cudnn.deterministic = False;   # 允许非确定性算法，提高性能
logging.info("✓ CUDA优化设置完成");

2025-12-27,15:51:37 | INFO | Rank 0 | ✓ CUDA优化设置完成


# 十二、加载教师模型（用于知识蒸馏）

In [75]:
if config.distillation:
    logging.info("步骤12: 加载教师模型（知识蒸馏）...")
else:
    logging.info("步骤12: 跳过教师模型加载（未启用知识蒸馏）")
if config.distillation:
    try:
        from modelscope.models import Model
    except:
        raise ImportError("modelscope is not installed. Please install it by `pip install modelscope`.")

    teacher_model_dict = {
        "damo/multi-modal_team-vit-large-patch14_multi-modal-similarity" : {"model": "image_model"},
        "damo/multi-modal_rleg-vit-large-patch14" : {"model": "encode_image"},
        "damo/multi-modal_clip-vit-huge-patch14_zh" : {"clip_model": "encode_image"},
        "damo/multi-modal_clip-vit-large-patch14_zh" : {"clip_model": "encode_image"},
    }
    assert config.teacher_model_name in teacher_model_dict, "Error: Valid teacher model name has not been built."

    try:
        teacher_model = Model.from_pretrained(config.teacher_model_name)
    except Exception as e:
        if "Unexpected key(s) in state_dict" in str(e):
            error_message = (
                "An error occurred while loading the model: {}\n"
                "Maybe you should update modelscope. ".format(e)
            )
            raise RuntimeError(error_message)

    for k, v in teacher_model.state_dict().items():
        v.requires_grad = False
    
    # mapping different extract_features function to same name
    mapping = teacher_model_dict[config.teacher_model_name]
    if "model" in mapping and hasattr(teacher_model, "model"):
        model_instance = getattr(teacher_model, "model")
        if hasattr(model_instance, mapping["model"]):
            setattr(teacher_model, "get_feature", getattr(model_instance, mapping["model"]))
    elif "clip_model" in mapping and hasattr(teacher_model, "clip_model"):
        model_instance = getattr(teacher_model, "clip_model")
        if hasattr(model_instance, mapping["clip_model"]):
            setattr(teacher_model, "get_feature", getattr(model_instance, mapping["clip_model"]))

    # 单节点训练：直接将教师模型移动到GPU，不使用DDP
    teacher_model = teacher_model.to(device)
    # 使用相同的 ModelWrapper 包装以兼容 train.py 中的 teacher_model.module 引用
    teacher_model = ModelWrapper(teacher_model)
    logging.info(f"Teacher model loaded from {config.teacher_model_name}")
    logging.info(f"✓ 教师模型加载完成: {config.teacher_model_name}")
else:
    teacher_model = None

2025-12-27,15:51:39 | INFO | Rank 0 | 步骤12: 跳过教师模型加载（未启用知识蒸馏）


# 十三、训练循环


In [76]:
!python --version


Python 3.10.19


In [77]:

# from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler
# ============================================================================
# Chinese-CLIP项目内部模块导入
# ============================================================================
from cn_clip.clip import load  # 模型权重加载函数
from cn_clip.clip.model import convert_weights, convert_state_dict, resize_pos_embed, CLIP  # CLIP模型定义
from cn_clip.training.train import train, evaluate  # 训练和评估函数
from cn_clip.training.data import get_data  # 数据加载器构建
from cn_clip.training.params import parse_args  # 命令行参数解析
from cn_clip.training.logger import setup_primary_logging, setup_worker_logging  # 日志系统设置
from cn_clip.training.scheduler import cosine_lr  # 余弦学习率调度器


logging.info(f"步骤14: 开始训练循环 (从epoch {start_epoch} 到 {config.max_epochs})...")
logging.info("========== 所有初始化完成，开始训练 ==========")


 

logging.debug(f"steps:{steps}, data:{data}");
 

# 从start_epoch开始训练，直到max_epochs
# start_epoch可能是0（从头训练）或从检查点恢复的epoch编号
for epoch in range(start_epoch, config.max_epochs):
    logging.info(f"========== 开始训练 Epoch {epoch + 1}/{config.max_epochs} ==========")
    
    # 记录当前epoch开始（只有主进程记录）
    #if is_master(args) == 0:
    logging.info(f'Start epoch {epoch + 1}')
    
    # 执行训练
    logging.info(f"开始执行训练 (epoch {epoch + 1})...")
    
    logging.info(f"model:{model}, data:{data}, epoch:{epoch}, \n optimizer:{optimizer}, scaler:{scaler}, \nscheduler:{scheduler}, args:{config}, steps:{steps}");
    # train函数会：
    #   1. 遍历训练数据加载器
    #   2. 前向传播计算损失（对比学习损失）
    #   3. 反向传播计算梯度
    #   4. 更新模型参数
    #   5. 记录训练指标（损失、准确率等）
    #   6. 返回本epoch的步数
    # 如果启用知识蒸馏，会传入teacher_model用于计算蒸馏损失
    if config.distillation:
        # 知识蒸馏训练：使用教师模型指导学生模型训练
        num_steps_this_epoch = train(model, data, epoch, optimizer, scaler, scheduler, config, steps, teacher_model)
    else:# 标准训练：只使用对比学习损失
        num_steps_this_epoch = train(model, data, epoch, optimizer, scaler, scheduler, config, steps);
    
        
    # 累计总步数（用于学习率调度和日志记录）
    steps += num_steps_this_epoch
    logging.info(f"✓ Epoch {epoch + 1} 训练完成，总步数: {steps}")

    # 执行验证（如果满足验证条件）
    logging.info("检查是否需要执行验证...")
    # 验证条件：
    #   1. 指定了验证数据集（val_data不为None）
    #   2. 指定了验证间隔（valid_epoch_interval不为None）
    #   3. 当前epoch满足验证间隔（(epoch + 1) % valid_epoch_interval == 0）
    if config.val_data is not None and config.valid_epoch_interval is not None and ((epoch + 1) % config.valid_epoch_interval) == 0:
        assert "val" in data, "Error: Valid dataset has not been built."
        
        # evaluate函数会：
        #   1. 在验证集上计算图像和文本特征
        #   2. 计算检索指标（R@1, R@5, R@10等）
        #   3. 记录验证结果到日志
        logging.info(f"开始执行验证 (epoch {epoch + 1})...")
        if not config.use_flash_attention:
            evaluate(model, data, epoch, config, steps)
        else:
            # FlashAttention需要FP16精度，使用autocast上下文管理器
            with torch.cuda.amp.autocast():
                evaluate(model, data, epoch, config, steps)
        logging.info(f"✓ 验证完成 (epoch {epoch + 1})")

    # 如果还有下一个epoch，为下一个epoch重新加载数据集和数据加载器
    # 这样可以支持每个epoch使用不同的数据增强策略
    # 例如：某些数据增强策略可能需要根据epoch动态调整
    if epoch + 1 < config.max_epochs:
        data = get_data(config, epoch_id=epoch + 1, max_txt_length=config.context_length)

    # ====================================================================
    # 步骤15: 保存检查点
    # ====================================================================
    # 只有主进程且本epoch有训练步数时才保存检查点
    if num_steps_this_epoch > 0:
        logging.info(f"步骤13: 保存检查点 (epoch {epoch + 1})...");
    if num_steps_this_epoch > 0:
        # 保存定期检查点（根据save_epoch_frequency或最后一个epoch）
        # 保存条件：
        #   1. 是最后一个epoch（epoch + 1 == max_epochs）
        #   2. 或者满足保存频率（(epoch + 1) % save_epoch_frequency == 0）
        # 定期检查点文件名：epoch{epoch+1}.pt，例如：epoch1.pt, epoch2.pt, epoch3.pt
        if (epoch + 1) == config.max_epochs or (
            config.save_epoch_frequency > 0 and ((epoch + 1) % config.save_epoch_frequency) == 0
        ):
            t1 = time.time()
            save_path = os.path.join(config.checkpoint_path, f"epoch{epoch + 1}.pt")
            
            # 保存检查点内容：
            #   - epoch: 当前epoch编号，用于恢复训练时确定起始epoch
            #   - step: 当前总步数，用于恢复训练时确定起始步数
            #   - name: 实验名称，用于标识检查点所属的实验
            #   - state_dict: 模型权重，包含所有参数的当前值
            #   - optimizer: 优化器状态，包含动量、学习率调度等信息
            # 注意：如果使用FlashAttention，需要转换state_dict格式
            torch.save(
                {
                    "epoch": epoch + 1,      # 当前epoch数
                    "step": steps,           # 当前总步数
                    "name": config.name,       # 实验名称
                    "state_dict": model.state_dict() if not config.use_flash_attention else convert_state_dict(model.state_dict()),  # 模型权重
                    "optimizer": optimizer.state_dict(),  # 优化器状态（包含Adam的动量等）
                },
                save_path,
            )
            logging.info("Saved checkpoint {} (epoch {} @ {} steps) (writing took {} seconds)".format(save_path, epoch + 1, steps, time.time() - t1))
            logging.info(f"✓ 定期检查点已保存: {save_path}")
        
        # 保存最新检查点（每个epoch都保存，用于自动恢复训练）
        # 文件名固定为：epoch_latest.pt
        # 如果训练中断，可以通过加载epoch_latest.pt自动恢复训练
        t1 = time.time()
        save_path = os.path.join(config.checkpoint_path, f"epoch_latest.pt")
        torch.save(
            {
                "epoch": epoch + 1,
                "step": steps,
                "name": config.name,
                "state_dict": model.state_dict() if not config.use_flash_attention else convert_state_dict(model.state_dict()),
                "optimizer": optimizer.state_dict(),
            },
            save_path,
        )
        logging.info("Saved checkpoint {} (epoch {} @ {} steps) (writing took {} seconds)".format(save_path, epoch + 1, steps, time.time() - t1))
        logging.info(f"✓ 最新检查点已保存: {save_path}")

2025-12-27,15:51:43 | INFO | Rank 0 | 步骤14: 开始训练循环 (从epoch 0 到 10)...
2025-12-27,15:51:43 | INFO | Rank 0 | ========== 所有初始化完成，开始训练 ==========
2025-12-27,15:51:43 | DEBUG | Rank 0 | steps:0, data:{'train': DataInfo(dataloader=<torch.utils.data.dataloader.DataLoader object at 0x0000028D9F50C520>, sampler=<torch.utils.data.sampler.RandomSampler object at 0x0000028D9F50D750>, dataset=<cn_clip.training.data.LMDBDataset object at 0x0000028D9F114D00>, epoch_id=0), 'val': DataInfo(dataloader=<torch.utils.data.dataloader.DataLoader object at 0x0000028D9F50DC90>, sampler=<torch.utils.data.sampler.RandomSampler object at 0x0000028D9F50FF70>, dataset=<cn_clip.training.data.LMDBDataset object at 0x0000028D9F4EB490>, epoch_id=0)}
2025-12-27,15:51:43 | INFO | Rank 0 | ========== 开始训练 Epoch 1/10 ==========
2025-12-27,15:51:43 | INFO | Rank 0 | Start epoch 1
2025-12-27,15:51:43 | INFO | Rank 0 | 开始执行训练 (epoch 1)...
2025-12-27,15:51:43 | INFO | Rank 0 | model:<model_wrapper.ModelWrapper object at 0x000

AttributeError: 'ConfigParam' object has no attribute 'should_save'